# Data Science Project SoSe 2024
## Team 07
- Maximilian Hoffmann
- Kilian Kempf
- Daniel Schneider
- Tom Schuck

## Submission of Task 2

In [191]:
import os
import pandas as pd

### Data Preparation

In [192]:
DATA_DIR = os.path.join(os.getcwd(), 'data/Instacart')

orders = pd.read_csv(os.path.join(DATA_DIR, 'orders.csv.zip'))

tip_train = pd.read_csv(os.path.join(DATA_DIR, 'tip_trainingsdaten1_.csv'))[['order_id', 'tip']]

In [193]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [194]:
orders_tip = pd.merge(orders, tip_train)
orders_tip

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip
0,2539329,1,prior,1,2,8,NaN,False
1,2398795,1,prior,2,3,7,15.0,False
2,473747,1,prior,3,3,12,21.0,False
3,2254736,1,prior,4,4,7,29.0,False
4,431534,1,prior,5,4,15,28.0,False
...,...,...,...,...,...,...,...,...
3214869,2558525,206209,prior,9,4,15,22.0,False
3214870,2266710,206209,prior,10,5,18,29.0,False
3214871,1854736,206209,prior,11,4,10,30.0,False
3214872,626363,206209,prior,12,1,12,18.0,False


In [195]:
orders_tip.loc[0, 'order_number']

1

### Task 2.1

In [196]:
round(orders_tip.shape[0] * 0.01)

32149

In [197]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

features = ['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']
param_grid = {'criterion': ['gini'],# 'entropy', 'log_loss'], 
              'max_depth': [depth for depth in range(1, len(features[:3]) + 1)],
              'min_samples_leaf': [2**i for i in range(0, 3)]}
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid)

In [198]:
X = orders_tip[features]
y = orders_tip[['tip']]
grid_search.fit(X, y)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini'], 'max_depth': [1, 2, 3],
                         'min_samples_leaf': [1, 2, 4]})

In [199]:
decision_tree_clf = grid_search.best_estimator_
grid_search.best_params_

{'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 2}

### Task 2.2

In [200]:
# estimated accuracy of the decision tree on the test set
grid_search.best_score_

0.6047997541104627

### Task 2.3

In [201]:
orders_tip['tip'].value_counts()

tip
False    1797616
True     1417258
Name: count, dtype: int64

In [202]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='most_frequent')
scores = cross_val_score(dummy, X, y, cv=5)

In [203]:
scores.mean()

0.5591559731424575

### Task 2.4

In [204]:
tip_test = pd.read_csv(os.path.join(DATA_DIR, 'tip_testdaten1_template.csv'))
tip_test

,Unnamed: 0,order_id,tip
0,10,1187899,NaN
1,25,1492625,NaN
2,49,2196797,NaN
3,74,525192,NaN
4,78,880375,NaN
...,...,...,...
131204,3420838,2585586,NaN
131205,3420862,943915,NaN
131206,3420924,2371631,NaN
131207,3420933,1716008,NaN


In [205]:
orders_tip_test = pd.merge(orders, tip_test)
orders_tip_test

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,Unnamed: 0,tip
0,1187899,1,train,11,4,8,14.0,10,NaN
1,1492625,2,train,15,1,11,30.0,25,NaN
2,2196797,5,train,5,0,11,6.0,49,NaN
3,525192,7,train,21,2,11,6.0,74,NaN
4,880375,8,train,4,1,14,10.0,78,NaN
...,...,...,...,...,...,...,...,...,...
131204,2585586,206199,train,20,2,16,30.0,3420838,NaN
131205,943915,206200,train,24,6,19,6.0,3420862,NaN
131206,2371631,206203,train,6,4,19,30.0,3420924,NaN
131207,1716008,206205,train,4,1,16,10.0,3420933,NaN


In [206]:
orders_tip_test['tip'] = decision_tree_clf.predict(orders_tip_test[features])
orders_tip_test

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,Unnamed: 0,tip
0,1187899,1,train,11,4,8,14.0,10,False
1,1492625,2,train,15,1,11,30.0,25,False
2,2196797,5,train,5,0,11,6.0,49,True
3,525192,7,train,21,2,11,6.0,74,False
4,880375,8,train,4,1,14,10.0,78,True
...,...,...,...,...,...,...,...,...,...
131204,2585586,206199,train,20,2,16,30.0,3420838,False
131205,943915,206200,train,24,6,19,6.0,3420862,False
131206,2371631,206203,train,6,4,19,30.0,3420924,False
131207,1716008,206205,train,4,1,16,10.0,3420933,True


In [207]:
orders_tip_test_csv = orders_tip_test[tip_test.columns].copy()
orders_tip_test_csv.rename(columns={tip_test.columns[0]:''}, inplace=True)
orders_tip_test_csv

,,order_id,tip
0,10,1187899,False
1,25,1492625,False
2,49,2196797,True
3,74,525192,False
4,78,880375,True
...,...,...,...
131204,3420838,2585586,False
131205,3420862,943915,False
131206,3420924,2371631,False
131207,3420933,1716008,True


In [208]:
orders_tip_test_csv.to_csv(os.path.join(DATA_DIR, 'tip_testdaten1.csv'), index=False)

### Task 2.5 a)

In [209]:
op_prior = pd.read_csv(os.path.join(DATA_DIR, 'order_products__prior.csv.zip'))
op_train = pd.read_csv(os.path.join(DATA_DIR, 'order_products__train.csv.zip'))
op = pd.concat([op_prior, op_train])
op

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [210]:
first_orders = orders[orders['order_number']==1]
first_orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
11,2168274,2,prior,1,2,11,NaN
26,1374495,3,prior,1,1,14,NaN
39,3343014,4,prior,1,6,11,NaN
45,2717275,5,prior,1,3,12,NaN
...,...,...,...,...,...,...,...
3420930,969311,206205,prior,1,4,12,NaN
3420934,3189322,206206,prior,1,3,18,NaN
3421002,2166133,206207,prior,1,6,19,NaN
3421019,2227043,206208,prior,1,1,15,NaN


In [211]:
first_orders_products = pd.merge(first_orders[['order_id']], op[['order_id', 'product_id']])
first_orders_products

,order_id,product_id
0,2539329,196
1,2539329,14084
2,2539329,12427
3,2539329,26088
4,2539329,26405
...,...,...
2078063,3154581,20073
2078064,3154581,13176
2078065,3154581,20842
2078066,3154581,23594


In [212]:
(first_orders_products[first_orders_products['product_id']==21137].shape[0] / first_orders_products.shape[0]) * 100

0.7922743625328912

### Task 2.5 b)

In [213]:
orders.head(12)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [214]:
last_orders = orders[['user_id', 'order_number']].groupby('user_id').max().reset_index()
last_orders

,user_id,order_number
0,1,11
1,2,15
2,3,13
3,4,6
4,5,5
...,...,...
206204,206205,4
206205,206206,68
206206,206207,17
206207,206208,50


In [215]:
last_orders = pd.merge(orders, last_orders, on=['user_id', 'order_number'])
last_orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1187899,1,train,11,4,8,14.0
1,1492625,2,train,15,1,11,30.0
2,2774568,3,test,13,5,15,11.0
3,329954,4,test,6,3,12,30.0
4,2196797,5,train,5,0,11,6.0
...,...,...,...,...,...,...,...
206204,1716008,206205,train,4,1,16,10.0
206205,1043943,206206,test,68,0,20,0.0
206206,2821651,206207,test,17,2,13,14.0
206207,803273,206208,test,50,5,11,4.0


In [216]:
last_orders_products = pd.merge(last_orders[['order_id']], op[['order_id', 'product_id']])
last_orders_products

,order_id,product_id
0,1187899,196
1,1187899,25133
2,1187899,38928
3,1187899,26405
4,1187899,39657
...,...,...
1384612,272231,40603
1384613,272231,15655
1384614,272231,42606
1384615,272231,37966


In [217]:
(last_orders_products[last_orders_products['product_id']==21137].shape[0] / last_orders_products.shape[0]) * 100

0.7867879709695893

##### Comparison:
Percentage of orders containing product 21137 (organic strawberries) first orders of the respective user: $\approx 0.792\%$\
Percentage of orders containing product 21137 (organic strawberries) last orders of the respective user: $\approx 0.787\%$

The difference can probably explained as being random, since it is relatively small.